In [ ]:
import boto3
import sagemaker

region = boto3.Session().region_name
role = sagemaker.get_execution_role()

pipeline_name = f"GLAIR-BCAF-Consultation-HPO" # Change these to reflect your project/business name

In [ ]:
from pipelines.CarPriceML_HPO.pipeline import get_pipeline


pipeline = get_pipeline(
    region=region,
    role=role,
    pipeline_name=pipeline_name
)

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
s3_singapore = boto3.client("s3", region_name="ap-southeast-1")

def get_latest_file(bucket_name, prefix_name):
    s3_uri_response = s3_singapore.list_objects_v2(Bucket=bucket_name, Prefix=prefix_name)
    latest_key = sorted(s3_uri_response.get("Contents", []), key=lambda x: x["LastModified"], reverse=True)[0]["Key"]

    return f"s3://{bucket_name}/{latest_key}"
    
s3_uri_lelang = get_latest_file(
    "glair-exploration-sagemaker-s3-bucket-singapore",
    "glair-bcaf-consultation-input/training/lelang"
)

s3_uri_crawling = get_latest_file(
    "glair-exploration-sagemaker-s3-bucket-singapore",
    "glair-bcaf-consultation-input/training/crawling"
)

In [ ]:
execution = pipeline.start(
    parameters=dict(
            ProcessingInstanceType="ml.m5.xlarge",
            ProcessingInstanceCount="1",
            TrainingInstanceType="ml.m5.xlarge",
            TrainingInstanceCount="1",
            InputDataLelangURI=s3_uri_lelang,
            InputDataCrawlingURI=s3_uri_crawling,
    )
)

In [ ]:
execution.describe()